# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = "../output_data/weather_data.csv"
weather_data = pd.read_csv(weather)
weather_data

,Unnamed: 0,City,Country,Temperatures (F),Latitude,Longitude,Humidity %,Wind (mph),Cloudiness
0,0,manzhouli,CN,6.35,49.6000,117.4333,95,5.06,96
1,1,khatanga,RU,-19.52,71.9667,102.5000,83,4.63,34
2,2,vaini,TO,82.40,-21.2000,-175.2000,89,9.22,75
3,3,nouadhibou,MR,64.76,20.9310,-17.0347,73,17.67,0
4,4,ranong,TH,74.19,9.9658,98.6348,87,4.38,41
...,...,...,...,...,...,...,...,...,...
547,547,kalcheva,UA,48.51,45.7374,28.8141,78,8.23,85
548,548,columbus,US,21.65,39.9612,-82.9988,93,8.05,90
549,549,tiznit,MA,54.66,29.5833,-9.5000,85,0.74,37
550,550,kuryk,KZ,45.63,43.2000,51.6500,66,12.53,28


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_data[['Latitude', 'Longitude']]
humidity = weather_data['Humidity %']
max_humidity = humidity.max()

In [4]:
fig = gmaps.figure(zoom_level = 2, center = (0,0))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = weather_data.loc[(weather_data['Temperatures (F)']<80) &
                                 (weather_data['Temperatures (F)']> 65)& 
                                 (weather_data['Humidity %']< 50) &
                                (weather_data['Cloudiness']< 60)]
ideal_weather

,Unnamed: 0,City,Country,Temperatures (F),Latitude,Longitude,Humidity %,Wind (mph),Cloudiness
58,58,ushuaia,AR,77.00,-54.8000,-68.3000,27,18.41,0
73,73,saint george,US,65.80,37.1041,-113.5841,25,6.91,1
111,111,fortuna foothills,US,74.70,32.6578,-114.4119,23,4.61,1
130,130,tessalit,ML,68.65,20.1986,1.0114,18,9.53,3
201,201,ancud,CL,78.17,-41.8697,-73.8203,47,6.91,0
256,256,cabo san lucas,MX,77.00,22.8909,-109.9124,37,3.00,0
275,275,laguna,US,67.57,38.4210,-121.4238,48,2.82,1
309,309,jalingo,NG,77.00,8.8833,11.3667,16,3.36,36
325,325,adrar,MR,65.59,20.5022,-10.0711,21,9.42,15
334,334,rapar,IN,66.99,23.5667,70.6333,40,8.12,59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_weather.filter(['City','Country'])
hotel_df['Hotel Name'] = ""
hotel_df['Lat'] = ""
hotel_df['Lng']= ""
hotel_df

,City,Country,Hotel Name,Lat,Lng
58,ushuaia,AR,,,
73,saint george,US,,,
111,fortuna foothills,US,,,
130,tessalit,ML,,,
201,ancud,CL,,,
256,cabo san lucas,MX,,,
275,laguna,US,,,
309,jalingo,NG,,,
325,adrar,MR,,,
334,rapar,IN,,,


In [23]:
no_nearby = []
for index, row in ideal_weather.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    coords=f'{latitude},{longitude}'
    try:
        params={ 
            "location":coords,
            "radius" : 5000,
            "type" :"lodging", 
            "key" : g_key
            }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response=requests.get(base_url, params=params).json()
        hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index,'Lat'] = response['results'][0]['geometry']['location']["lat"]
        hotel_df.loc[index,'Lng'] = response['results'][0]['geometry']['location']["lng"]
    except:
        print(f'No nearby hotels to {city}.')

No nearby hotels to tessalit.
No nearby hotels to adrar.
No nearby hotels to ati.


In [33]:
#Remove cities with no nearby hotels
hotel_df = hotel_df.loc[hotel_df['Hotel Name']!= ""]
hotel_df

,City,Country,Hotel Name,Lat,Lng
58,ushuaia,AR,Albatros Hotel,-54.807,-68.3049
73,saint george,US,Best Western Plus Abbey Inn,37.089,-113.584
111,fortuna foothills,US,Fortuna De Oro RV Park,32.6683,-114.398
201,ancud,CL,Hotel Arena Gruesa,-41.8604,-73.8224
256,cabo san lucas,MX,Hotel Tesoro Los Cabos,22.8809,-109.912
275,laguna,US,Holiday Inn Express & Suites Elk Grove Central...,38.4241,-121.399
309,jalingo,NG,Debright Motel,8.882,11.3738
334,rapar,IN,Dharamshi Bapa’s Home,23.5707,70.6423
353,atar,MR,Oumou Elghoura,20.5224,-13.0504
362,bilma,NE,Expeditions Ténére voyages,18.693,12.9177


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotels= hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotels)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))